In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from pymatgen.core.periodic_table import _pt_data

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert
)
from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
OPENAI_GPT_MODEL = "gpt-4"

In [2]:
import re

mp_llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4-1106-preview",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=5,
    # streaming=True
)

llm = ChatOpenAI(
    temperature=0.7,
    model="gpt-4",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    MPSynthesisExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # ArxivAgent(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    # arxiv,
    # wikipedia,
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        # SystemMessage(content=re.sub(
        #     r"\s+", " ",
        #     """You are a helpful data-aware agent that can consult materials-related
        #     data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
        #     user to clarify their queries if needed. Please note that you don't have 
        #     direct control to MP but through multiple assistant agents to help you. 
        #     You need to provide complete context for them to do their job. 
        #     """).replace("\n", " ")
        # )
        ],
    # "early_stopping_method": 'generate',
    # "extra_prompt_messages": 
    # )
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

# agent_executor = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     max_iterations=5,
# )

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
agent_executor.invoke({
    "input": "How to synthesis LiFePO4? Please provide a detailed step-by-step procedure and the reference.",
})



> Entering new AgentExecutor chain...


The assistant should use the MPSynthesisExpert tool to find the detailed step-by-step procedure and the reference for synthesizing LiFePO4.
Action:
```
{
  "action": "MPSynthesisExpert",
  "action_input": "How to synthesis LiFePO4?"
}
```

> Entering new AgentExecutor chain...


/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```
{
  "action": "search_materials_synthesis__get",
  "action_input": {
    "target_formula": "LiFePO4"
  }
}
```{"target_formula": "LiFePO4"}


Retrieving SynthesisSearchResultModel documents:   0%|          | 0/285 [00:00<?, ?it/s]

...

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```
{
  "action": "search_materials_synthesis__get",
  "action_input": {
    "target_formula": "LiFePO4",
    "limit": 5,
    "fields": "references"
  }
}
```{"target_formula": "LiFePO4", "limit": 5, "fields": "references"}


Retrieving SynthesisSearchResultModel documents:   0%|          | 0/285 [00:00<?, ?it/s]

return first 5 synthesis results for now
[{'doi': '10.1021/jp407510h', 'paragraph_string': '6Li3Fe3(PO4)3 was prepared by the previously reported high-temperature solid-state synthesis as ...', 'synthesis_type': 'solid-state', 'reaction_string': '9 Fe2O3 + 18 LiOH + 18 NH4H2PO4 == 1 Li3Fe3(PO4)3 + 36 H2O + 18 NH3 + 4.5 O2', 'reaction': {'left_side': [{'amount': '18', 'material': 'NH4H2PO4'}, {'amount': '18', 'material': 'LiOH'}, {'amount': '9', 'material': 'Fe2O3'}], 'right_side': [{'amount': '1', 'material': 'Li3Fe3(PO4)3'}, {'amount': '36', 'material': 'H2O'}, {'amount': '18', 'material': 'NH3'}, {'amount': '4.5', 'material': 'O2'}]}, 'targets_formula': ['Li18 Fe18 P18 O72'], 'target': {'material_string': '6Li3Fe3(PO4)3', 'material_name': '', 'material_formula': 'Li3Fe3(PO4)3', 'phase': None, 'is_acronym': False, 'composition': [{'formula': 'Li3Fe3(PO4)3', 'amount': '6', 'elements': {'P': '3', 'O': '12', 'Li': '3', 'Fe': '3'}}], 'amounts_vars': {}, 'elements_vars': {}, 'additives': [

{'input': 'How to synthesis LiFePO4? Please provide a detailed step-by-step procedure and the reference.',
 'chat_history': [],
 'output': 'LiFePO4 can be synthesized using various methods. One method involves a high-temperature solid-state synthesis using precursors such as Fe2O3, LiOH, and NH4H2PO4. The precursors are mixed and heated at temperatures up to 850°C for several days. Another method uses a sol-gel combustion approach with precursors like FePO4 and Li2CO3, where the mixture is calcined at 700°C in a nitrogen atmosphere. For detailed procedures, please refer to the following scientific literature: \n\n1. High-temperature solid-state synthesis - DOI: 10.1021/jp407510h\n2. Ion-exchange method - DOI: 10.1021/cm402617b\n3. Solution method - DOI: 10.1021/jp306936t\n4. Solid-state reactions - DOI: 10.1021/cm303259j\n5. Sol-gel combustion method - DOI: 10.1007/s10853-011-6139-7'}

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

LiFePO4 can be synthesized using various methods. One method involves a high-temperature solid-state synthesis using precursors such as Fe2O3, LiOH, and NH4H2PO4. The precursors are mixed and heated at temperatures up to 850°C for several days. Another method uses a sol-gel combustion approach with precursors like FePO4 and Li2CO3, where the mixture is calcined at 700°C in a nitrogen atmosphere. For detailed procedures, please refer to the following scientific literature: 

1. High-temperature solid-state synthesis - DOI: 10.1021/jp407510h
2. Ion-exchange method - DOI: 10.1021/cm402617b
3. Solution method - DOI: 10.1021/jp306936t
4. Solid-state reactions - DOI: 10.1021/cm303259j
5. Sol-gel combustion method - DOI: 10.1007/s10853-011-6139-7


In [5]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    # streaming=True
)

response = llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


In [6]:
print(response.content)

Synthesizing LiFePO4 (lithium iron phosphate) typically involves a solid-state reaction method. Below is a step-by-step procedure for synthesizing LiFePO4:

Materials:
- Lithium carbonate (Li2CO3)
- Iron (II) acetate (Fe(CH3COO)2)
- Ammonium dihydrogen phosphate (NH4H2PO4)
- Phosphoric acid (H3PO4)
- Deionized water
- Ethanol
- Mortar and pestle
- Crucible
- Furnace

Procedure:
1. Weigh out the appropriate amounts of lithium carbonate (Li2CO3) and iron (II) acetate (Fe(CH3COO)2) in a 1:1 molar ratio. For example, if you want to synthesize 1 mole of LiFePO4, you would weigh out 74.55 grams of Li2CO3 and 173.93 grams of Fe(CH3COO)2.

2. Grind the lithium carbonate and iron (II) acetate together using a mortar and pestle to ensure a homogenous mixture.

3. Weigh out the appropriate amount of ammonium dihydrogen phosphate (NH4H2PO4) and add it to the lithium carbonate and iron (II) acetate mixture in a 1:1 molar ratio. For example, if you are synthesizing 1 mole of LiFePO4, you would weigh